# Amin Saeidi - 400211579

# Question1

In [1]:
# First import essential packagess
using COBREXA
using GLPK # to As Optimizer
using JuMP

#  Lets Go!

In [4]:
# Here we first define a function in which model will prepare

function loading_model(model_name)
    
    model_file_name = string(model_name,".json")
    download_link = string("http://bigg.ucsd.edu/static/models/", model_file_name)
    
    println("MODEL-NAME: ", model_name)

    !isfile(model_file_name) && download(download_link, model_file_name)
    myModel = load_model(StandardModel, model_file_name) # we specifically want to load a StandardModel from the model file    
    return myModel
    
end

loading_model (generic function with 1 method)

In [5]:
# For e-coli-core
e_coli_main_model = loading_model("e_coli_core")

MODEL-NAME: e_coli_core


Metabolic model of type StandardModel
sparse([9, 51, 55, 64, 65, 34, 44, 59, 66, 64  …  20, 22, 23, 25, 16, 17, 34, 44, 57, 59], [1, 1, 1, 1, 1, 2, 2, 2, 2, 3  …  93, 93, 94, 94, 95, 95, 95, 95, 95, 95], [1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 1.0  …  1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0], 72, 95)
Number of reactions: 95
Number of metabolites: 72


# Homogenization

In [6]:
# Homogenization

M = 10 ^ 6

for i in 1:n_reactions(e_coli_main_model)
    reaction_id = reactions(e_coli_main_model)[i]
    if(e_coli_main_model.reactions[reaction_id].lb >= 0)
        e_coli_main_model.reactions[reaction_id].lb = 0
        e_coli_main_model.reactions[reaction_id].ub = M 
    else
        e_coli_main_model.reactions[reaction_id].lb = -M
        e_coli_main_model.reactions[reaction_id].ub = M
    end
end

# Question1-Part1-Single-Reaction-Deletion

In [7]:
# The main method for finding essential reactions without using cobrexa modifications

function single_deletion(model)
    
#     essential_reactions = Dict()
    essential_reactions = []
    e_coli_biomass = 0.8739 # From Cobrexa
    first_FBA_dict = flux_balance_analysis_dict(model, GLPK.Optimizer)
    f = 0.01
    
    for i in 1:n_reactions(model)
        reaction_id = reactions(model)[i]
        lb = model.reactions[reaction_id].lb
        ub = model.reactions[reaction_id].ub
        model.reactions[reaction_id].lb = 0
        model.reactions[reaction_id].ub = 0
        temp_dict = flux_balance_analysis_dict(model, GLPK.Optimizer)
        a = keys(flux_summary(temp_dict).biomass_fluxes)
        b = []
        for key in a
            append!(b,(flux_summary(temp_dict).biomass_fluxes)[key])
        end
        temp_v_biomass = b[1]
        if  temp_v_biomass <= (f * e_coli_biomass)
#             essential_reactions[i] = reaction_id
               append!(essential_reactions,[reaction_id])
        end
        model.reactions[reaction_id].lb = lb
        model.reactions[reaction_id].ub = ub
    end 
    
    return essential_reactions
end

single_deletion (generic function with 1 method)

In [9]:
# The main method for finding essential reactions - using cobrexa modifications

function single_deletion(model)
    
    essential_reactions = Dict()
    first_FBA_dict = flux_balance_analysis_dict(model, GLPK.Optimizer)
    e_coli_biomass = first_FBA_dict["BIOMASS_Ecoli_core_w_GAM"]
    f = 0.01
    
    for i in 1:n_reactions(model)
        reaction_id = reactions(model)[i]
        restricted_flux = flux_balance_analysis_dict(
            model,
            GLPK.Optimizer,
            modifications = [change_constraint(reaction_id, lb = 0, ub = 0.0)],
        );
        temp_v_biomass = restricted_flux["BIOMASS_Ecoli_core_w_GAM"]
        if  temp_v_biomass <= (f * e_coli_biomass)
            essential_reactions[i] = reaction_id
        end
    end 
    
    return essential_reactions
end

single_deletion (generic function with 1 method)

In [8]:
single = single_deletion(e_coli_main_model)

18-element Vector{Any}:
 "PGK"
 "PGM"
 "PIt2r"
 "ACONTa"
 "ACONTb"
 "BIOMASS_Ecoli_core_w_GAM"
 "CS"
 "RPI"
 "ENO"
 "EX_glc__D_e"
 "EX_h_e"
 "EX_nh4_e"
 "EX_pi_e"
 "GAPD"
 "GLCpts"
 "GLNS"
 "ICDHyr"
 "NH4t"

# Question1-Part2-Double-Reaction-Deletion

In [9]:
# The main method for finding double essential reactions without using cobrexa modifications

function double_deletion(model,single)
    
    single_values = values(single)
    essential_reactions = []
#     essential_reactions = Dict()
    e_coli_biomass = 0.8739 # From Cobrexa
    first_FBA_dict = flux_balance_analysis_dict(model, GLPK.Optimizer)
    f = 0.01
    
    for i in 1:n_reactions(model)
        for j in i+1:n_reactions(model)
            reaction_id_i = reactions(model)[i]
            reaction_id_j = reactions(model)[j]
            
            lb_i = model.reactions[reaction_id_i].lb
            ub_i = model.reactions[reaction_id_i].ub
            lb_j = model.reactions[reaction_id_j].lb
            ub_j = model.reactions[reaction_id_j].ub
            
            model.reactions[reaction_id_i].lb = 0
            model.reactions[reaction_id_i].ub = 0
            model.reactions[reaction_id_j].lb = 0
            model.reactions[reaction_id_j].ub = 0
            
            temp_dict = flux_balance_analysis_dict(model, GLPK.Optimizer)
            a = keys(flux_summary(temp_dict).biomass_fluxes)
            b = []
            for key in a
                append!(b,(flux_summary(temp_dict).biomass_fluxes)[key])
            end
            temp_v_biomass = b[1]
            if  (temp_v_biomass <= (f * e_coli_biomass)) & (reaction_id_i ∉ single_values) & (reaction_id_j ∉ single_values)
#                 essential_reactions[reaction_id_i] = reaction_id_j
                append!(essential_reactions, [(reaction_id_i,reaction_id_j)])
            end
            model.reactions[reaction_id_i].lb = lb_i
            model.reactions[reaction_id_i].ub = ub_i
            model.reactions[reaction_id_j].lb = lb_j
            model.reactions[reaction_id_j].ub = ub_j
        end
    end 
    
    return essential_reactions
end

double_deletion (generic function with 1 method)

In [10]:
double_deletion(e_coli_main_model,single)

111-element Vector{Any}:
 ("PFK", "PGI")
 ("PFK", "PGL")
 ("PFK", "CO2t")
 ("PFK", "RPE")
 ("PFK", "CYTBD")
 ("PFK", "TALA")
 ("PFK", "TKT1")
 ("PFK", "TKT2")
 ("PFK", "EX_co2_e")
 ("PFK", "EX_h2o_e")
 ("PFK", "EX_o2_e")
 ("PFK", "G6PDH2r")
 ("PFK", "GND")
 ⋮
 ("EX_h2o_e", "FORt")
 ("EX_o2_e", "FBA")
 ("FBA", "G6PDH2r")
 ("FBA", "GND")
 ("FBA", "H2Ot")
 ("FBA", "NADH16")
 ("FBA", "NADTRHD")
 ("FBA", "O2t")
 ("FORt", "H2Ot")
 ("FORt", "NADH16")
 ("FORt", "PDH")
 ("GLUDy", "GLUSy")

# Question2

In [11]:
# Blocked reactions; from previous HW
blocked_reactions = ["EX_fum_e","EX_fru_e","FRUpts2","EX_mal__L_e","FUMt2_2","MALt2_2","GLNabc","EX_gln__L_e"];

# Reactions for which we want to find coupling condition
Q_reactions = ["ACONTb","GAPD","CS","PGK","ENO","GLCpts","PGM","EX_glc__D_e","BIOMASS_Ecoli_core_w_GAM","ACONTa"];

In [12]:
# Removing blocked reactions
function remove_blocked(model, blocked_reactions)
    for reaction in blocked_reactions
        remove_reaction!(model, reaction)
    end
    return model
end

remove_blocked (generic function with 1 method)

In [13]:
revised_model = remove_blocked(e_coli_main_model, blocked_reactions)

Metabolic model of type StandardModel
sparse([9, 51, 55, 64, 65, 34, 44, 59, 66, 64  …  20, 22, 23, 25, 16, 17, 34, 44, 57, 59], [1, 1, 1, 1, 1, 2, 2, 2, 2, 3  …  85, 85, 86, 86, 87, 87, 87, 87, 87, 87], [1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 1.0  …  1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0], 72, 87)
Number of reactions: 87
Number of metabolites: 72


In [14]:
# finding block reactions
# Defining the main method
function find_blocks(model)
    
    n = n_reactions(model)
    S = stoichiometry(model)
    
    irreversible_reactions = Dict()
    reversible_reactions = Dict()
    Keys = keys(model.reactions)
    index = 1  # Fucking Index
    for key in Keys
        reaction_id = key
        if(model.reactions[reaction_id].lb >= 0)
            irreversible_reactions[index] = reaction_id
        else
            reversible_reactions[index] = reaction_id
        end
        index = index + 1
    end
    
    
    lb = []
    ub = []
    for i in 1:n
        reaction_id = reactions(model)[i]
        append!(lb, model.reactions[reaction_id].lb)
        append!(ub, model.reactions[reaction_id].ub)  
    end
    
    tolerance = 10^(-8)
    
    blocked_reactions = []
    checker = Dict()

    re = 0
    irre = 0
    # For ire : ni
    for i in keys(irreversible_reactions)

        # Define the Model
        solver = Model(GLPK.Optimizer)

        # Create Variables
        @variable(solver,lb[i] <= v[i = 1:n] <= ub[i])

        # Specify Constraints
        @constraint(solver, (S)*v .== 0)
        @constraint(solver, v[i] <= 1)
        @objective(solver,Max,v[i])

        # Solving/Optimize
        optimize!(solver)

        if (-tolerance <= objective_value(solver) <= tolerance)
            append!(blocked_reactions,[irreversible_reactions[i]])
            irre += 1
        end
    end

    # For re : 2 * nr
    for i in keys(reversible_reactions)

        # Define the Model
        solver = Model(GLPK.Optimizer)

        # Create Variables
        @variable(solver,lb[i] <= v[i = 1:n] <= ub[i])

        # Specify Constraints
        @constraint(solver, (S)*v .== 0)    
        @constraint(solver, v[i] <= 1)
        @objective(solver,Max,v[i])

        # Solving/Optimize
        optimize!(solver)

        if (-tolerance <= objective_value(solver) <= tolerance)
            checker[i] = reversible_reactions[i]
        end
    end

    for i in keys(reversible_reactions)

        # Define the Model
        solver = Model(GLPK.Optimizer)

        # Create Variables
        @variable(solver,lb[i] <= v[i = 1:n] <= ub[i])

        # Specify Constraints
        @constraint(solver, (S)*v .== 0)    
        @constraint(solver, v[i] >= -1)
        @objective(solver,Min,v[i])

        # Solving/Optimize
        optimize!(solver)

        if (-tolerance <= objective_value(solver) <= tolerance) && reversible_reactions[i] in values(checker)
            append!(blocked_reactions,[reversible_reactions[i]])
            re += 1
        end
    end
    return blocked_reactions
end    

find_blocks (generic function with 1 method)

In [17]:
# Main fucntion: using FBA to find coupling relations

function coupling(model,Q_reactions)
    n_reactions = length(Q_reactions)
    matrix = Matrix{Float64}(undef,n_reactions,n_reactions)   
    
    # Matrix notation 
    # i_j : 3  (v_j = 0 => v_i = 0)
    # partially_coupled : 2
    # fully_coupled : 1
    # un_coupled : 0
    
    for i in 1:n_reactions
        for j in i:n_reactions 
            
            reaction_id_i = Q_reactions[i]
            reaction_id_j = Q_reactions[j]
            
            lb_i = model.reactions[reaction_id_i].lb
            ub_i = model.reactions[reaction_id_i].ub
            model.reactions[reaction_id_i].lb = 0
            model.reactions[reaction_id_i].ub = 0
            blocks_i = find_blocks(model);;
            model.reactions[reaction_id_i].lb = lb_i
            model.reactions[reaction_id_i].ub = ub_i
            
            lb_j = model.reactions[reaction_id_j].lb
            ub_j = model.reactions[reaction_id_j].ub
            model.reactions[reaction_id_j].lb = 0
            model.reactions[reaction_id_j].ub = 0
            blocks_j = find_blocks(model);;
            model.reactions[reaction_id_j].lb = lb_j
            model.reactions[reaction_id_j].ub = ub_j
            if i == j
                matrix[i,j] = 1
            else   
                if reaction_id_i in blocks_j
                    matrix[i,j] = 3
                end

                if reaction_id_j in blocks_i
                    matrix[j,i] = 3
                end
                
                if (reaction_id_i in blocks_j) & (reaction_id_j in blocks_i)
                    matrix[i,j] = 2
                    matrix[j,i] = 2
                end

                if (matrix[i,j] == 2) & (matrix[j,i] == 2)
                    temp_flux = flux_balance_analysis_dict(model, GLPK.Optimizer)
                    flux_i = temp_flux[reaction_id_i]
                    flux_j = temp_flux[reaction_id_j]
                    if ((flux_i % flux_j) == 0) | ((flux_j % flux_i) == 0)
                        matrix[i,j] = 1
                        matrix[j,i] = 1
                    end
                end
                
                if !(matrix[i,j] in [1,2,3])
                    matrix[i,j] = 0
                end
                if !(matrix[j,i] in [1,2,3])
                    matrix[j,i] = 0
                end
                
            end
        end
    end
    return matrix
end

coupling (generic function with 1 method)

In [18]:
coupling(revised_model, Q_reactions)

10×10 Matrix{Float64}:
 1.0  3.0  2.0  3.0  3.0  3.0  3.0  3.0  0.0  2.0
 0.0  1.0  0.0  2.0  2.0  3.0  2.0  3.0  0.0  0.0
 2.0  3.0  1.0  3.0  3.0  3.0  3.0  3.0  0.0  2.0
 0.0  2.0  0.0  1.0  2.0  3.0  2.0  3.0  0.0  0.0
 0.0  2.0  0.0  2.0  1.0  3.0  1.0  3.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0
 0.0  2.0  0.0  2.0  1.0  3.0  1.0  3.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0
 3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  1.0  3.0
 2.0  3.0  2.0  3.0  3.0  3.0  3.0  3.0  0.0  1.0